In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as ipw

from ipywidgets import interact

from bokeh.models import ColumnDataSource, Legend, LegendItem
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from collections import OrderedDict
old_settings = np.seterr(over='ignore') #Ignore warnings about overflow data points

### First we define the relevant physical constants such as Boltzmann constant, initial temperature, etc

In [ ]:
# Set up constant
N = 200;
kB = 8.617e-5; #eV/K
T = 0.001; # Boltzmann constant measured interms of chemical potential
# currenT = [f'T = {T:.2f} K']*N; #Create an array of labels to initiate interactive legend
# currenT2 = [f'T = {T:.2f} K']*N;
currenT = f'T = {T:.2f} K';
currenT2 = f'T = {T:.2f} K';
kBT = kB * T;
E_min = -10.0;
E_max = 10.0; # Energy in eV
factor = 0.2;

# Set up the variable space for 1D plot
En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
f = 1.0/(np.exp( (factor*En_mu)/kBT )+1) # Occupation number according to F-D statistics
df = np.gradient(f) # Take the first derivative of the F-D distribution.

### Next, we initiate two figure objects, and each with two plots

In [ ]:
# Set up the first interactive 1D plot
plot = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 1D",
              tools="pan,reset,save,wheel_zoom",
              x_range=[E_min, E_max], y_range=[0.0, 1.1])
plot.xaxis[0].axis_label='Energy relative to chemical potential, E - \u03BC (eV)';
plot.yaxis[0].axis_label='Fermi-Dirac distribution f(E)';
source = ColumnDataSource(dict(x=En_mu, y=f));
source1 = ColumnDataSource(dict(x=En_mu, y=f));
figure_1 = plot.line(x='x',y='y', source=source, line_color = 'blue', line_width=3, line_alpha=0.6);
figure_2 = plot.line(x='x',y='y', source=source1, line_color = 'red', line_width=3, line_alpha=0.6);
legend_1 = Legend(items = [LegendItem(label=currenT, renderers=[figure_2]), LegendItem(label=currenT2, renderers=[figure_1])]);
plot.add_layout(legend_1);

In [ ]:
# Set up an interactive 1D plot of the first derivative of the F-D distribution at low temperature
plot3 = figure(plot_height=400, plot_width=400, title="First derivative",
              tools="pan,reset,save,wheel_zoom",
              x_range=[E_min, E_max], y_range=[0.0, 0.6])
plot3.xaxis[0].axis_label='Energy relative to chemical potential, E - \u03BC (eV)';
plot3.yaxis[0].axis_label='First derivative df(E)/dE [1/eV]';
source3 = ColumnDataSource(dict(x=En_mu, y=-df));
source4 = ColumnDataSource(dict(x=En_mu, y=-df));
figure_3 = plot3.line(x='x', y='y', source=source3, line_color = 'blue', line_width=3, line_alpha=0.6);
source4 = ColumnDataSource(dict(x=En_mu, y=-df));
figure_4 = plot3.line(x='x', y='y', line_color = 'red', source=source4, line_width=3, line_alpha=0.6);
legend_2 = Legend(items = [LegendItem(label=currenT, renderers=[figure_4]), LegendItem(label=currenT2, renderers=[figure_3])]);
plot3.add_layout(legend_2);

### Then we define a function to call each time we want to update the figures

In [ ]:
# Set up callbacks to update the 1D live graph
def update_data(T,TT):
    # Generate the new curve
    kBT = kB*T;
    kBT2 = kB*TT;
    currenT = f'T = {T:.2f} K';
    currenT2 = f'T = {TT:.2f} K';
    En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
    figure_2.data_source.data['y'] = 1.0/(np.exp( (factor*En_mu)/kBT )+1);
    figure_1.data_source.data['y'] = 1.0/(np.exp( (factor*En_mu)/kBT2 )+1);
    Ex,Ey = np.meshgrid(np.linspace(-E_max,E_max,2*N), np.linspace(-E_max,E_max,2*N));
    Er = np.sqrt(Ex**2 + Ey**2); 
    figure_4.data_source.data['y'] = -np.gradient(figure_2.data_source.data['y']);
    figure_3.data_source.data['y'] = -np.gradient(figure_1.data_source.data['y']);
    legend_1.items = [LegendItem(label=currenT, renderers=[figure_2]), LegendItem(label=currenT2, renderers=[figure_1])];
    legend_2.items = [LegendItem(label=currenT, renderers=[figure_4]), LegendItem(label=currenT2, renderers=[figure_3])];
    push_notebook();

### Last but not least, show the figures and interactive tool to change temperatures, each time it triggers a call to the function to update the figure.

In [ ]:
show(row(plot,plot3),notebook_handle=True);
interact(update_data, T = ipw.FloatSlider(min=0.001,max=1000.001,step=10,value=300,description = 'Temp (K)'), TT = ipw.FloatSlider(min=0.001,max=1000.001,step=10,value=0.001,description = 'Temp (K)'));